# In this notebook we display the performance of the final version of damped Newton with preconditioning.

In [ ]:
from __future__ import division
import os 
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/DampedNewtonPreconditioningFinal_images'):
    os.makedirs('../Images/DampedNewtonPreconditioningFinal_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp(x, col, plt, scale = 200, edgecolors = "k"):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors, c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
N = [ 1000, 1100 ]

In [ ]:
x = np.random.rand( 2, N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r = 0.8 + .2 * np.random.rand( 1, N[1] )
y = np.vstack( ( r * np.cos(theta), r * np.sin(theta) ) )

In [ ]:
import computational_OT

## Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\varepsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence and $\ \mathcal{U}(\alpha,\beta)=\{\pi: \pi\mathcal{1}=\alpha, \pi^{T}\mathcal{1}=\beta\}$.

# 0. Sinkhorn vs damped Newton

## 0.1. Sinkhorn algorithm 
The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = \alpha \odot diag(u)K diag(v)\odot \beta
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following alternative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ ,\ 
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\varepsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$



In [ ]:
# Sinkhorn
print("Sinkhorn... ")
print("Doing for (",N[0], N[1],").")
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
epsilons = [ 0.1, 0.5, 0.05, 0.03, 0.02, 0.010 ]
Pmatrix_dist_linVSsinkhorn = []
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  print( "For epsilon: ", str(eps) )
  #Kernel
  K = np.exp( - C/eps )
  print( " |- Iterating")
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 1000)
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP.append( P_opt )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range(len(results_Sinkhorn)):
  error = np.asarray( results_Sinkhorn[i]['error_a'] ) + np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( loc = 'upper right' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/ConvergenceSinkhornvaryingepsilon.pdf", format = 'pdf')
plt.show()

# 0.2. Damped Newton-Raphson

## Damped Newton algorithm:
The dual formulation of OT is given by
$$
OT_{\varepsilon} = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

The Hessian is given by 
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)=\frac{-1}{\varepsilon}
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\ , \ $ where $\pi\mathbb{1}_{m} = \alpha,\ \pi^{T}\mathbb{1}_{n}=\beta,\ $ and $\Delta = diag: \mathbb{R}^{n} \rightarrow M_{n}(\mathbb{R})$ is the linear operator mapping a vector  to a diagonal matrix  containing  this vector.


This implies 
$$
\begin{pmatrix}
\Delta(\alpha) && \pi_{\varepsilon}\\
\pi^{T}_{\varepsilon} && \Delta(\beta) 
\end{pmatrix}
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix} = 0\ ,
$$
that is,
$$
\begin{pmatrix}
\mathbb{1}_{n}\\
\mathbb{1}_{m}
\end{pmatrix}\in \ker(\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g))\ .
$$
Hence, $\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)$ is singular. Therefore, on regularization we have the following Hessian
$
H_{reg} := \nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)+\lambda cc^{T}\ ,
$ 
where $c= \begin{pmatrix}\frac{\mathbb{1}}{\sqrt{n+m}}\\-\frac{\mathbb{1}}{\sqrt{n+m}}\end{pmatrix}\in M_{(n+m),1}(\mathbb{R})$.

Now, at the $k^{th}$ iteration solve
$\nabla^{2}Q_{\alpha, \beta,\varepsilon}(f,g)p_{k} = \nabla Q_{\alpha, \beta,\varepsilon}(f,g)$ to obtain the optimizing direction vector $p_{k}$ and then perform the Armijo condition to obtain the update step $\alpha_{k}$ such that we have the update
$$
(f,g) \leftarrow (f,g) + \alpha_{k} p_{k}\ .
$$

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
withoutprecond_dampedNewtonP = []
withoutprecond_results_dampedNewton  = []
withoutprecond_times_dampedNewton    = []
withoutprecond_Hessians_dampedNewton = []
withoutprecond_epsilons = [ 1.0, 0.5, 0.1, 0.05, 0.03, 0.02, 0.01 ]
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in withoutprecond_epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( -C/eps )
    f, g = a, b
    print( " |- Iterating")  
    start = time.time()
    Optimizer = computational_OT.damped_Newton(     K,
                                                    a,
                                                    b,
                                                    f,
                                                    g,
                                                    eps,
                                                    rho,
                                                    c )
    out = Optimizer._update(    max_iterations = 30,
                                debug = False )
    end = time.time()
    if out != -1:
        withoutprecond_results_dampedNewton.append( out )
        withoutprecond_times_dampedNewton.append( end - start )        
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        withoutprecond_dampedNewtonP.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        withoutprecond_Hessians_dampedNewton.append( mat )
    else:
        withoutprecond_epsilons.remove( eps )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" ) 
for i in range( len( withoutprecond_results_dampedNewton ) ):
  error = np.asarray( withoutprecond_results_dampedNewton[i]['error_a'] ) + np.asarray( withoutprecond_results_dampedNewton[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = 'upper right' )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/ErrorDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) ) 
plt.title( "$$" )
plt.title( "Objective Function" )
for i in range( len( withoutprecond_results_dampedNewton ) ):
  plt.plot( np.asarray( withoutprecond_results_dampedNewton[i]['objective_values'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Objective value" )
plt.legend( loc = 'upper right' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/ObjectiveDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

### Ascent step-size plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Alpha" )
for i in range( len( withoutprecond_results_dampedNewton ) ):
  plt.plot( np.asarray( withoutprecond_results_dampedNewton[i]['linesearch_steps'] ), label = 'Damped Newton for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" )
plt.legend( loc = 'upper right' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/AlphaDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

# I. Plotting spectrum as a function of $\varepsilon$

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ :, sorting_indices ]    
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range(len(withoutprecond_epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon = "+str(eps))
    ev = spectral_decomposition( withoutprecond_Hessians_dampedNewton[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
plt.rcParams.update( { 'font.size': 10 } )
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows = len(withoutprecond_epsilons), ncols = 1, sharey = True )
plt.title("Histogram of eigenvalues.")
for i in range(len(withoutprecond_epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(withoutprecond_epsilons[i]))
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace  = 0.8 )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/eigenhistunstabilizedDampedNewtonwithoutPrecond.pdf", format = 'pdf' )
plt.show()

## Damped Newton with Preconditioning
Here we perform dual damped Newton with preconditioning. Here we consider $t$ eigenvalues of the Hessian that we want to move to one and form the following preconditioning matrix using the corresponding eigenvectors,
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ ,
$$
where
$$
y_{i} \in \ker\left(\nabla^{2}_{f}Q_{\alpha, \beta, \varepsilon}(f)-\lambda_{i}I_{n}\right),\ \forall i= 1,\dots,k\ ,
$$
 are orthonormal.

Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\alpha, \beta, \varepsilon}(f)P)(Pp_{k})=P\nabla Q_{\alpha, \beta, \varepsilon}(f)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the ascent direction $p_{k}$, following which we use the Armijo condition to obtain the ascent step size $\alpha_{k}$.

# II. Actual preconditioning

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ :, sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _, m = eigenvectors.shape
    indices=[]
    for i in range(num_eigs//2):
        indices.append( m - i - 2 )
        indices.append( i + 1 )
    # end for
    if num_eigs%2 != 0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[:, indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[ : , index ] )
    # end for
    return null_vector, precond_vectors

## Final version

### Exact Inversion

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton with preconditioning and exact inversion... ")
print( "Doing for (",N[0], N[1],")." )
# Get null vector and preconditioning vectors
num_eigs = 30 
null_vector, precond_vectors = build_preconditioners( num_eigs, withoutprecond_Hessians_dampedNewton[- 1], ansatz = False )
# Experiment
## Here EI: Exact inversion
rho = 0.95
c = 0.1
reset_starting_point = True
withprecond_final_modified_Hessians_EI = []
withprecond_dampedNewtonP_EI = []
withprecond_results_dampedNewton_EI  = []
withprecond_times_dampedNewton_EI    = []
withprecond_epsilons =  [ 1.0, 0.5, 0.05, 0.03, 0.02, 0.01 ]
f, g = None, None
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in withprecond_epsilons:
    print( "For epsilon = "+str(eps)+":" )  
    #Kernel
    K = np.exp( - C/eps )
    if ( f is None ) or ( g is None ): 
        f, g = a, b
    print( "Doing for (",N[0], N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = -1,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    withprecond_results_dampedNewton_EI.append( out )
    end = time.time()
    withprecond_times_dampedNewton_EI.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    withprecond_dampedNewtonP_EI.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:] 
    withprecond_final_modified_Hessians_EI.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len( withprecond_results_dampedNewton_EI ) ):
  error = np.asarray( withprecond_results_dampedNewton_EI[i]['error_a'] ) + np.asarray( withprecond_results_dampedNewton_EI[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend( loc = 'upper right' )
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/ErrorDampedNewtonwithPrecond_final_exact_inversion.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Average time plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(withprecond_results_dampedNewton_EI[0]['timings'][0]) ):
  values = []
  for i in range( len(withprecond_results_dampedNewton_EI) ):
    mean = 0
    for k in range( len(withprecond_results_dampedNewton_EI[i]['timings']) ):
      mean += withprecond_results_dampedNewton_EI[i]['timings'][k][j]
    # end for
    mean = mean/len( withprecond_results_dampedNewton_EI[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(withprecond_epsilons) == len(values):
    plt.plot( withprecond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = 'upper right' )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/Timings_DampedNewtonwithPrecond_final_exact_inversion.pdf", format = 'pdf' )
plt.show()

### Iterative inversion

In [ ]:
# Damped Newton with preconditioning
print( "Damped Newton with preconditioning and iterative inversion... " )
print( "Doing for (",N[0], N[1],")." )
# Get null vector and preconditioning vectors
num_eigs = 30 
null_vector, precond_vectors = build_preconditioners( num_eigs, withoutprecond_Hessians_dampedNewton[- 1], ansatz = False )
# Experitment
## Here I_inv: Iterative inversion
rho = 0.95
c = 0.1
reset_starting_point = True
withprecond_final_modified_Hessians_I_inv = []
withprecond_dampedNewtonP_I_inv = []
withprecond_results_dampedNewton_I_inv  = []
withprecond_times_dampedNewton_I_inv    = []
withprecond_epsilons =  [ 1.0, 0.5, 0.05, 0.03, 0.02, 0.01 ]
iter_inv = 30
r_tol = 1e-5
a_tol = 1e-10
f, g = None, None
# Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in withprecond_epsilons:
    print( "For epsilon: ", str(eps) )
    #Kernel
    K = np.exp( - C/eps )
    if ( f is None ) or ( g is None ): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = iter_inv,
                                relative_tol = r_tol,
                                absolute_tol = a_tol,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    withprecond_results_dampedNewton_I_inv.append( out )
    end = time.time()
    withprecond_times_dampedNewton_I_inv.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    withprecond_dampedNewtonP_I_inv.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:] 
    withprecond_final_modified_Hessians_I_inv.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len( withprecond_results_dampedNewton_I_inv ) ):
  error = np.asarray( withprecond_results_dampedNewton_I_inv[i]['error_a'] ) + np.asarray( withprecond_results_dampedNewton_I_inv[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/ErrorDampedNewtonwithPrecond_final_iterative_inversion.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

#### Average time plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]
plt.figure( figsize = ( 20, 10 ) )  
for j in range( len( withprecond_results_dampedNewton_I_inv[0]['timings'][0]) ):
  values = []
  for i in range( len(withprecond_results_dampedNewton_I_inv) ):
    mean = 0
    for k in range( len(withprecond_results_dampedNewton_I_inv[i]['timings']) ):
      mean += withprecond_results_dampedNewton_I_inv[i]['timings'][k][j]
    # end for
    mean = mean/len( withprecond_results_dampedNewton_I_inv[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(withprecond_epsilons) == len(values):
    plt.plot( withprecond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = 'upper right' )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/Timings_DampedNewtonwithPrecond_final_iterative_inversion.pdf", format = 'pdf' )
plt.show()

### Time to converge
The following plot displays a comparison between the exact inversion and the iterative inversion methods with preconditioning as a function of $\epsilon$.

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 20, 7 ) )   
plt.title( "$$" ) 
plt.title( "Time plot" ) 
plt.plot( withprecond_times_dampedNewton_EI[::-1], label = 'Damped Newton with true preconditioning and exact inversion with 30 preconditioning vectors $', linewidth = 2, marker = 'o' ) 
plt.plot( withprecond_times_dampedNewton_I_inv[::-1], label = 'Damped Newton with true preconditioning and CG with (num_precond, iter_inv, atol, rtol) = ('+str(num_eigs)+', '+str(iter_inv)+', '+str(a_tol)+',  '+str(r_tol)+') ', linewidth = 2, marker = 'o' ) 
plt.xlabel( " $\epsilon$ " )  
plt.ylabel( " Time in sec" ) 
plt.legend( loc = 'upper right' ) 
plt.savefig( "../Images/DampedNewtonPreconditioningFinal_images/Timeplot_EI_vs_II.pdf", format ='pdf' ) 
plt.show() 

# III. More spectral statistics

In [ ]:
def print_spectral_statistics( mat, stabilize = False ):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( (np.ones(N[0]), - np.ones(N[1])) )
        max_vector = np.hstack( (np.ones(N[0]),  np.ones(N[1])) )
        norm = np.sqrt( N[0] + N[1] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( ( min_vector.shape[0], 1 ) )
        max_vector = max_vector.reshape( ( max_vector.shape[0], 1 ) )
        #
        mat = mat + np.dot( min_vector, min_vector.T )
        mat = mat - np.dot( max_vector, max_vector.T )
    # endif
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]
    
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[ : 10 ])
    print( "List of largest  eigenvalues: ", eig[ - 10 : ])
    min_index = np.argmin( eig )
    max_index = np.argmax( eig )
    min_value = eig[ min_index ]
    max_value = eig[ max_index ]
    min_vector = v[ : , min_index ]
    min_vector = min_vector/min_vector[0]
    max_vector = v[ : , max_index ]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print( "Condition number: ", condition_number )
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range( len( withprecond_epsilons ) ):
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = withoutprecond_Hessians_dampedNewton[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
# Using the largest epsilon's eigenvectors for conditioning
print( "Building preconditioning eigenvectors"  )
#null_vector, precond_vectors = build_preconditioners( Hessians_DampedNewton[-1], ansatz=False )
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, withoutprecond_Hessians_dampedNewton[-1], ansatz = False )
for i in range( len( withprecond_epsilons ) ):
    eps = withprecond_epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = withprecond_final_modified_Hessians_I_inv[i]
    print_spectral_statistics( Hessian )
    print("")
# end for

In [ ]:
# Using the previous epsilon's eigenvectors for conditioning
print( "Building preconditioning eigenvectors"  )
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, withoutprecond_Hessians_dampedNewton[-1], ansatz = False )
for i in range( len( withprecond_epsilons ) ):
    eps = withprecond_epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    Hessian = withprecond_final_modified_Hessians_I_inv[i]
    print_spectral_statistics( Hessian )
    print( "" )
    print( "Building preconditioning eigenvectors" )
    num_eigs = 13
    null_vector, precond_vectors = build_preconditioners( num_eigs, withoutprecond_Hessians_dampedNewton[-1], ansatz = False )
    print( "" ) 
# end for